# Text Mining of BBC News Data

## Part 2: Bag-of-Words Text Vectorization


## The Analyzer Object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
count_vectorizer = CountVectorizer()
count_vectorizer

In [ ]:
test_sentence = "C'est l'été au Brésil!"

In [ ]:
word_analyzer = CountVectorizer().build_analyzer()
word_analyzer(test_sentence)

In [ ]:
word_analyzer = CountVectorizer(strip_accents="unicode", ngram_range=(2, 2)).build_analyzer()
word_analyzer(test_sentence)

In [ ]:
word_analyzer = CountVectorizer(strip_accents="unicode", ngram_range=(1, 2)).build_analyzer()
word_analyzer(test_sentence)

In [ ]:
word_analyzer = CountVectorizer(ngram_range=(2, 2)).build_analyzer()
word_analyzer(test_sentence)

## Analyzer = Preprocessor + Tokenizer (+ Token Filtering) (+ n-grams extraction)

In [ ]:
vectorizer = CountVectorizer(strip_accents="unicode", lowercase=True)

In [ ]:
test_sentence = "C'est l'été au Brésil!"

### Exercises

- Type `vectorizer.build_<TAB>` to see the list of methods of the vectorizer object;

- Use the vectorizer to build a preprocessor object and apply it to the test sentence: which transformations do you notice?

- Use the vectorizer to build a tokenizer object and apply it to the preprocessed test sentence from the previous step;

- Compare the results of the previous two steps with the output of the analyzer applied to the original test sentence.

In [ ]:
# %load notebook_solutions/build_preprocessor.py

In [ ]:
# %load notebook_solutions/build_tokenizer.py

## Vectorization of a Full Dataset

In [ ]:
from pathlib import Path


bbc_folder_path = Path("bbc")
text_filepaths = sorted(bbc_folder_path.glob("*/*.txt"))

Instead of decoding the text manually as we did before, we will path the filenames directly to the vectorizer and let it decode using the encoding of our choice and ignore decoding errors (as we did previously):

In [ ]:
%%time

vectorizer = CountVectorizer(encoding="utf-8", input="filename",
                             decode_error="ignore")

vectorizer.fit(text_filepaths)

In [ ]:
type(vectorizer.vocabulary_)

In [ ]:
len(vectorizer.vocabulary_)

In [ ]:
vocabulary_items = sorted(vectorizer.vocabulary_.items())

In [ ]:
vocabulary_items[:10]

In [ ]:
vocabulary_items[5000:5010]

In [ ]:
vocabulary_items[-10:]

In [ ]:
vectorized_docs = vectorizer.transform(text_filepaths)

In [ ]:
vectorized_docs

In [ ]:
first_doc = vectorized_docs[0, :]

In [ ]:
first_doc

In [ ]:
first_doc.toarray()

**Question**: why does the vectorizer return a sparse matrix instead of a regular (dense) array?

In [ ]:
first_doc.toarray().shape

Let's have a look at the indices of the non-zero values in the vectorized first document:

In [ ]:
first_doc.indices

Each dimension corresponds to a word in the vocabulary. We can retrieve the "feature name" for a specific dimension number as follows:

In [ ]:
feature_names = vectorizer.get_feature_names()
feature_names[:10]

In [ ]:
feature_names[5360]

It's possible to inverse a full vectorized document at once:

In [ ]:
vectorizer.inverse_transform(first_doc)

**Exercise**

Print the text of the original first document. And compare to the "inversed transformed" document (can you find the same words?):

**Question**:

- Why do you thing the result of the text vectorization procedure is called "Bag-of-Words" representation?

- What as the main limitation of the "Bag-of-Words" representation?

- Can you come up with a pair of sentence with completely different meanings but that would share the same Bag of Words vector?

## TF-IDF Normalization

- TF-IDF stands for **"Term Frequency" - "Inverse Document Frequency"**.

- **Term Frequency** is the number of times a term or token appears in a given document;

- **Document Frequency** is the number of times a term or token appears across all the documents of the corpus.


**Note**: depending on the context, frequencies can either be:

- **absolute** values (**integer counts**) or
- **relative** values (**floating point numbers between 0 and 1** for **ratio** between two quantities).


TF-IDF is a normalization procedures that transforms integer term frequency counts into a reweights term frequencies such that very frequent words (such as "the", "a", "and", "it"... in English) have lower importance than rarer words.

The general idea, is that if we note:

- $n_{t, d}$ the absolute term frequency (the number of times terms $t$ appears in document $d$);

- $N_D$ the total number of documents in the dataset $D$;

- $n_{t}$ the number of documents in dataset $D$ that contain the term $t$;

The **term frequency** component can either be defined as:

- $\mathrm{tf} (t,d) = n_{t, d}$

or my taking a subliner function of the counts such as:

- $\mathrm{tf} (t,d) = log(1 + n_{t, d})$

Then the raw **inverse document frequency** is:

- $\mathrm{idf} (t) = \frac{N}{n_{t}}$

In practice one often takes the log of this quantity and define:

- $\mathrm{idf} (t, D) = log(\frac{N_D}{n_{t}})$

The final TF-IDF value is computed by taking the product of those two quantities:

$$\mathrm{tfidf} (t,d,D) = \mathrm{tf} (t,d)\cdot \mathrm{idf} (t,D)$$

There are many variants and the one implemented in scikit-learn is not necessarily the most standard (for historical reasons). The Wikipedia article has the list of variants:

https://en.wikipedia.org/wiki/Tf–idf

The scikit-learn documentation has comprehensive information on the vectorizers of scikit-learn and the TF-IDF variant implemented there:

https://scikit-learn.org/stable/modules/feature_extraction.html

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer


tfidf_transformer = TfidfTransformer()
tfidf_docs = tfidf_transformer.fit_transform(vectorized_docs)
tfidf_docs

In [ ]:
first_doc.toarray()

In [ ]:
first_tfidf_doc = tfidf_docs[0, :]
first_tfidf_doc.toarray()

In [ ]:
tfidf_transformer.idf_

**Exercises**:
    
Using the `vectorizer.get_feature_names()` vector and the `tfidf_transformer.idf_`, compute the list of the top 10 least and most informative words in the BBC corpus.

Hint: you can put the two vectors in a `pandas.DataFrame` and use the [nlargest](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.nlargest.html) and [nsmallest](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.nsmallest.html) methods.

In [ ]:
# %load notebook_solutions/top_idf.py

**Questions**

- Why do the top 10 IDF-weighted terms have the same IDF value in your opinion?

- What is the problem with using such terms in a text classification scenario?

- What do you think about the top 10 lowest? What could we do to improve the computational performance of a text classifier?

## Vectorizing and Weighting at Once

In [ ]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer(encoding="utf-8", input="filename",
                                   decode_error="ignore",
                                   min_df=5, max_df=0.8)

tfidf_docs_trimmed = tfidf_vectorizer.fit_transform(text_filepaths)

In [ ]:
tfidf_docs_trimmed

**Questions**:
    
- What do you notice about the shape of this new vectorized text corpus (compared to the previous version)?

In [ ]:
import pandas as pd


weighted_terms_trimmed = pd.DataFrame({
    "term": tfidf_vectorizer.get_feature_names(),
    "idf": tfidf_vectorizer.idf_})

print("Most 'informative' terms:")
print(weighted_terms_trimmed.nlargest(10, "idf"), end="\n\n")

print("Least informative terms:")
print(weighted_terms_trimmed.nsmallest(10, "idf"), end="\n\n")

## Document Similarities in TF-IDF Space

A common way to measure the semantic similarity of two documents is to compute the cosine similarity of their TF-IDF vectors:


$$cosine(x_1, x_2) = \frac{x_1 \cdot x_2}{||x_1|| ||x_2||}$$


**Question**

- Show that if two vectors have unit norms, then maximizing their cosine similarity is equivalent to minimizing their euclidean distances.

In [ ]:
vectorized_docs.multiply(vectorized_docs)

In [ ]:
import numpy as np


def sparse_row_norms(data):
    return np.asarray((data.multiply(data)).sum(axis=1)).ravel()


sparse_row_norms(vectorized_docs)

In [ ]:
sparse_row_norms(tfidf_docs)

In [ ]:
sparse_row_norms(tfidf_docs_trimmed)

**Question**

- Why do all TF-IDF vectorized documents have unit norm? Hint: look at the default parameters for thoe classes.

In [ ]:
def sparse_dot_products(query_vector, other_vectors):
    dot_products = query_vector.multiply(other_vectors).sum(axis=1)
    return np.asarray(dot_products).ravel()

In [ ]:
similarities = sparse_dot_products(tfidf_docs[0, :], tfidf_docs[1:, :])

In [ ]:
categories = [path.parent.name for path in text_filepaths]

**Exercises**:
- Find the document id and category for the 15 documents that are most similar to the first document in the corpus.
- Print the text of the first document (the query) and the text of the most similar documnt from your computed list.

In [ ]:
# %load notebook_solutions/tfidf_similarities.py

## Impact of TF-IDF Weighting on K-NN Accuracy

In [ ]:
%%time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

n_neighbors = 5
cv_scores = cross_val_score(KNeighborsClassifier(n_neighbors=n_neighbors),
                            vectorized_docs, categories, cv=10)

print(f"Val. accuracy: {cv_scores.mean():.3f} (+/-{cv_scores.std():.3f})")

In [ ]:
%%time
cv_scores = cross_val_score(KNeighborsClassifier(n_neighbors=n_neighbors),
                            tfidf_docs, categories, cv=10)

print(f"Val. accuracy: {cv_scores.mean():.3f} (+/-{cv_scores.std():.3f})")

In [ ]:
%%time
cv_scores = cross_val_score(KNeighborsClassifier(n_neighbors=n_neighbors),
                            tfidf_docs_trimmed, categories, cv=10)

print(f"Val. accuracy: {cv_scores.mean():.3f} (+/-{cv_scores.std():.3f})")

**Questions**:

- Why is K-NN classification accuracy better on tf-idf vectors than on pure tf vectors (count vectors)?

- Why is K-NN classification speed is faster on trimmed tf-idf vectors than on the original tf-idf vectors?

- Why is K-NN classification accuracy better on trimmed tf-idf vectors than on the original tf-idf vectors?